In [ ]:
import pandas as pd
import folium 
import seaborn
import os
import json
import gzip
import requests

In [ ]:
folium.__version__

# 01 - Unemployment in European countries

In [ ]:
m_eu = folium.Map([47,8.3], zoom_start=4)
eu_topo_path = 'topojson/europe.topojson.json'
folium.TopoJson(open(eu_topo_path),
                'objects.europe',
                name='topojson',
               ).add_to(m_eu)
m_eu

# 02 - Unemployment in Swiss cantons

In [ ]:
m_ch = folium.Map([47,8.3], zoom_start=8)
canton_topo_path = 'topojson/ch-cantons.topojson.json'
#m_ch.geo_json(geo_path=canton_topo_path, topojson='objects.cantons')
#m_ch.geo_json

folium.TopoJson(open(canton_topo_path),
                'objects.cantons',
                name='topojson',
               ).add_to(m_ch)
m_ch

In [ ]:
resp = requests.get('http://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=data/tipsun10.tsv.gz')

In [ ]:
resp.text